<a href="https://colab.research.google.com/github/KHRMNKY/HuggingFace_Models_and_Spaces/blob/main/Sumarizer_Youtube_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
import gradio as gr
import fasttext
from huggingface_hub import hf_hub_download
from transformers import pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration


In [3]:
api_key= "#"

def youtube_url(url):
  transcript = ""
  list=url.split("=")
  video_id = list[1]
  list = YouTubeTranscriptApi.get_transcript(video_id, languages=['tr', 'en',"de"])
  for dict in list:
      transcript += dict["text"]
  return transcript


In [4]:
def chat(prompt):

  system_msg = """
  you are a youtube transcript summarizer.
  """
  client = OpenAI(api_key = api_key, base_url="https://api.deepseek.com/v1")

  completion = client.chat.completions.create(
  model="deepseek-chat",
  messages=[
      {"role": "system", "content": system_msg},
      {"role": "user", "content": prompt}
  ]
  )

  answer = completion.choices[0].message.content
  return answer


In [8]:
def translate(answer):

  model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
  model = fasttext.load_model(model_path)

  detect_language = model.predict(answer[0:50])

  if detect_language[0][0]=='__label__eng_Latn' :

    tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
    model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
    input_ids = tokenizer(f"translate this part {answer[0:50]} to Turkish", return_tensors="pt").input_ids

    translated_ids = model.generate(input_ids)
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translated_text

  else:
    return answer


In [9]:
def main(url):
  transcript = youtube_url(url)
  query=f"summarize this {transcript} transcript"
  answer = chat(query)
  translated_text = translate(answer)
  print(type(translated_text))
  return translated_text

main("https://www.youtube.com/watch?v=01sAkU_NvOY")

BadRequestError: Error code: 400 - {'detail': 'Content Exists Risk'}

In [ ]:
def main(url):
  transcript = youtube_url(url)
  query=f"summarize this {transcript} transcript"
  answer = chat(query)
  translated_text = translate(answer)
  return translated_text


iface = gr.Interface(
    fn=main,
    inputs="text",
    outputs="text",

    title="Video Özeti Oluşturucu",
    description="Bir YouTube videosunun URL'sini girin ve özetini alın."
)

iface.launch(share = True)
